In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50) 
pd.set_option('display.max_columns', 50) 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# append multiple csv files to singular file

In [2]:
# create an empty pandas data frame
df_nonstrict = pd.DataFrame()

dir = r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\dfs_nonstrict"
 
# iterate over all files within folder
for file in sorted(os.listdir(dir), key=len):
    if file.endswith(".csv"):
        df_nonstrict = pd.concat([df_nonstrict , pd.read_csv(os.path.join(dir, file), sep=";", decimal=",", encoding='unicode_escape' )], axis=0 )
df_nonstrict.reset_index(drop=True, inplace=True) # reset the index 
df_nonstrict = df_nonstrict.drop(["index"], axis=1)

# add suffix to last three columns
new_names = [(i,i+"_nonstrict") for i in df_nonstrict.iloc[:, -3:].columns.values]
df_nonstrict.rename(columns = dict(new_names), inplace=True)

df_nonstrict['date_announced'] = pd.to_datetime(df_nonstrict['date_announced'])

# df_nonstrict
 
## save
path = Path.home() / 'Desktop' 
path_with_time = Path.joinpath(path, f'{time.strftime("%m%d")}_Eikon_LinksYahoo_nonstrict_9496_firms.csv')
df_nonstrict.to_csv(path_with_time, sep=";", decimal=",", index=False)
print(f"{path_with_time} saved")

C:\Users\JLU-SU\Desktop\1101_Eikon_LinksYahoo_nonstrict_9496_firms.csv saved


# calc cosine similarty

In [3]:
df_raw = pd.read_csv(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\Archive\1101_Eikon_Links_9496_firms.csv", sep=";", decimal=",", encoding='unicode_escape')

#### regex cheat sheet: 
https://www.debuggex.com/cheatsheet/regex/python  
https://regex101.com/r/3btjRd/4

### convert links to firm names

In [4]:
df = df_raw.copy()

## set links containing SRCH to nan since they appear to be always wrong
df = df.replace(
    {r'(^.*SRCH.*$)': np.nan, 
    r"^.*\/NA\.htm.*": np.nan, #https://www.glassdoor.com/Reviews/NA.htm
    r"^.*\/index\.htm.*": np.nan, #https://www.glassdoor.com/Reviews/index.htm
    r"^.*Glassdoor-Reviews-E100431.*": np.nan}, #https://www.glassdoor.com/Reviews/Glassdoor-Reviews-E100431.htm
    regex=True) 


## insert three empty columns after the link columns to extract the firm name from the gd link
for index, column in enumerate(["link_marius", "link_hannes", "link_hannes_nonstrict"], start=1):
    df.insert(df.columns.get_loc("link_hannes_nonstrict")+index, column.replace("link", "firm"), np.nan)
    # copy links in the empty new columns
    df[column.replace("link", "firm")] = df[column]
    # remove the https://www.glassdoor.com/Reviews/ part
    df[column.replace("link", "firm")] = df[column.replace("link", "firm")].str.split('/').str[-1]

## convert links to clean firm names
# https://www.glassdoor.com/Reviews/International-Specialty-Products-Reviews-E1114.htm
# to
# International Specialty Products
df[["firm_marius", "firm_hannes", "firm_hannes_nonstrict"]] = df[["firm_marius", "firm_hannes", "firm_hannes_nonstrict"]].replace(
    {"-Reviews|Working-at-":"", 
    r"(?:(-E\d.*)|(-E.*I_.*)\.htm.*$)": "",
    "-": " ",
    "#NAME?": np.nan}, 
    regex=True)

# df["link_found"].unique().tolist()
# wrong link - kann man vergessen
# df = df[df["link_found"]=="transformed jobs link"].reset_index(drop=True)

df.head()

,eikon_deal_id,date_announced,acquiror,high_quality_observation,total_assets,full_name,full_name_cleaned,full_name_legal_addon,permid,isin,...,link_hannes_nonstrict,firm_marius,firm_hannes,firm_hannes_nonstrict,review_amount_marius,review_amount_hannes,review_amount_hannes_nonstrict,link_found_hannes,link_found_hannes_nonstrict,cosine_marius
0,34391733382,07/01/2008,1,1,6.856333e+08,NuVista Energy Ltd,NuVista Energy,Ltd,4.295861e+09,CA67072Q1046,...,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,NaN,NuVista Energy,NuVista Energy,NaN,8.0,8.0,original review link,original review link,NaN
1,34391733368,07/01/2008,1,1,1.408352e+09,L-1 Identity Solutions Inc,L-1 Identity Solutions,Inc,4.295904e+09,US4519061018,...,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,NaN,NaN,Identity Solutions,NaN,NaN,NaN,no link found,transformed overview link,NaN
2,34391297298,08/01/2008,1,1,9.573430e+08,MutualFirst Financial Inc,MutualFirst Financial,Inc,4.295914e+09,US62845B1044,...,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,NaN,NaN,Mutual of Omaha,NaN,NaN,805.0,no link found,transformed overview link,NaN
3,34391297367,09/01/2008,1,1,5.611700e+08,Hampton Roads Bankshares Inc,Hampton Roads Bankshares,Inc,4.295902e+09,US9841022026,...,https://www.glassdoor.com/Reviews/United-Bankshares-Reviews-E1987.htm,Xenith Bankshares,Hampton Roads Bankshares,United Bankshares,1.0,NaN,162.0,original review link,transformed overview link,0.281349
4,34391297484,10/01/2008,1,1,3.081413e+09,Employers Holdings Inc,Employers,Holdings Inc,4.295903e+09,US2922181043,...,https://www.glassdoor.com/Reviews/EMPLOYERS-Reviews-E25702.htm,Wells Fargo,EMPLOYERS,EMPLOYERS,39130.0,253.0,253.0,original review link,original review link,NaN


low priority:
high_quality == 0
dreimal no link found
unsere reviews amounts im bereich +/- 10% -> review_certainty = 0 -> low certainty

3M fehlt weil tetragramme


### string_grouper: calculate cosine similarity

In [36]:
from string_grouper import compute_pairwise_similarities

df_similarity = df.copy()

## compute_pairwise_similarities method cant handle np.nan -> use empty strings
df_similarity[["firm_marius", "firm_hannes", "firm_hannes_nonstrict"]] = df_similarity[["firm_marius", "firm_hannes", "firm_hannes_nonstrict"]].replace(np.nan, "")

for column in ["firm_marius", "firm_hannes", "firm_hannes_nonstrict"]:
    df_similarity[column.replace("firm", "similarity")] = round(
        compute_pairwise_similarities(df_similarity["full_name_cleaned"], df_similarity[column], ignore_case=True, ngram_size=3),
        2)

## get the mean of the three cosine scores
df_similarity["similarity_mean"] = df_similarity[["similarity_marius", "similarity_hannes", "similarity_hannes_nonstrict"]].mean(axis=1)

## check whether firm name of marius is equal to hannes
df_similarity["marius_equalto_hannes"] = np.where( (df_similarity["firm_marius"] == df_similarity["firm_hannes"]) | (df_similarity["firm_marius"] == df_similarity["firm_hannes_nonstrict"]),  1, 0)

## get the firm name of the max cos value
# insert new column
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+1, "best_match_name", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+2, "best_match_review_amount", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+3, "best_match_cosine", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+4, "best_match_link", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+5, "best_match_type", np.nan)
# create list with column names with the highest similarity value
best_matches_names = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "firm"}, regex=True).tolist() #change the "similarity" part of the best_match item to fit the firm_column, eg similarity_hannes -> firm_hannes column
best_matches_review_amount = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "review_amount"}, regex=True).tolist()
best_matches_similarity = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).tolist()
best_matches_links = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "link"}, regex=True).tolist()
best_matches_types = [i.split("_")[-1:] for i in best_matches_names]
for index, columnname in enumerate(best_matches_names):
    df_similarity.loc[index, "best_match_name"] = df_similarity.loc[index, columnname]
    df_similarity.loc[index, "best_match_review_amount"] = df_similarity.loc[index, best_matches_review_amount[index]]
    df_similarity.loc[index, "best_match_cosine"] = df_similarity.loc[index, best_matches_similarity[index]]
    df_similarity.loc[index, "best_match_link"] = df_similarity.loc[index, best_matches_links[index]]
    df_similarity.loc[index, "best_match_type"] = best_matches_types[index][0]

df_similarity[:4]

,eikon_deal_id,date_announced,acquiror,high_quality_observation,total_assets,full_name,full_name_cleaned,best_match_name,best_match_review_amount,best_match_link,best_match_cosine,full_name_legal_addon,permid,isin,is_a_serial_acquiror_or_serial_target,link_marius,link_hannes,link_hannes_nonstrict,firm_marius,firm_hannes,firm_hannes_nonstrict,review_amount_marius,review_amount_hannes,review_amount_hannes_nonstrict,link_found_hannes,link_found_hannes_nonstrict,cosine_marius,similarity_marius,similarity_hannes,similarity_hannes_nonstrict,similarity_mean,marius_equalto_hannes
0,34391733382,07/01/2008,1,1,6.856333e+08,NuVista Energy Ltd,NuVista Energy,NuVista Energy,8.0,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,1.00,Ltd,4.295861e+09,CA67072Q1046,0,NaN,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,,NuVista Energy,NuVista Energy,NaN,8.0,8.0,original review link,original review link,NaN,0.00,1.0,1.00,0.666667,0
1,34391733368,07/01/2008,1,1,1.408352e+09,L-1 Identity Solutions Inc,L-1 Identity Solutions,Identity Solutions,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,0.83,Inc,4.295904e+09,US4519061018,1,NaN,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,,,Identity Solutions,NaN,NaN,NaN,no link found,transformed overview link,NaN,0.00,0.0,0.83,0.276667,1
2,34391297298,08/01/2008,1,1,9.573430e+08,MutualFirst Financial Inc,MutualFirst Financial,Mutual of Omaha,805.0,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,0.36,Inc,4.295914e+09,US62845B1044,0,NaN,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,,,Mutual of Omaha,NaN,NaN,805.0,no link found,transformed overview link,NaN,0.00,0.0,0.36,0.120000,1
3,34391297367,09/01/2008,1,1,5.611700e+08,Hampton Roads Bankshares Inc,Hampton Roads Bankshares,Hampton Roads Bankshares,NaN,https://www.glassdoor.com/Reviews/Hampton-Roads-Bankshares-Reviews-E24277.htm,1.00,Inc,4.295902e+09,US9841022026,1,https://www.glassdoor.com/Reviews/Xenith-Bankshares-Reviews-E38765.htm,https://www.glassdoor.com/Reviews/Hampton-Roads-Bankshares-Reviews-E24277.htm,https://www.glassdoor.com/Reviews/United-Bankshares-Reviews-E1987.htm,Xenith Bankshares,Hampton Roads Bankshares,United Bankshares,1.0,NaN,162.0,original review link,transformed overview link,0.281349,0.31,1.0,0.35,0.553333,0


In [13]:
## save file with dynamic file name
dir = Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample')
path = Path.joinpath(dir, f'{time.strftime("%m%d%y")}_GoogleAndYahoo_Cosine.csv')
df_similarity.to_csv(path, sep=";", decimal=",", index=False)
print(f"{path} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\110922_GoogleAndYahoo_Cosine.csv saved


# get best result

In [30]:
df_similarity = pd.read_csv(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\111422_BingAndYahoo_9496_complete.csv", sep=";", decimal=",", encoding='unicode_escape')

## get the firm name of the max cos value
# insert new column
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+1, "best_match_name", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+2, "best_match_review_amount", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+3, "best_match_cosine", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+4, "best_match_link", np.nan)
df_similarity.insert(df_similarity.columns.get_loc("full_name_cleaned")+5, "best_match_type", np.nan)
# create list with column names with the highest similarity value
best_matches_names = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "firm"}, regex=True).tolist() #change the "similarity" part of the best_match item to fit the firm_column, eg similarity_hannes -> firm_hannes column
best_matches_review_amount = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "review_amount"}, regex=True).tolist()
best_matches_similarity = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).tolist()
best_matches_links = df_similarity[["similarity_hannes", "similarity_hannes_nonstrict", "similarity_marius"]].idxmax(axis=1).replace({"similarity": "link"}, regex=True).tolist()
best_matches_types = [i.split("_")[-1:] for i in best_matches_names]
for index, columnname in enumerate(best_matches_names):
    df_similarity.loc[index, "best_match_name"] = df_similarity.loc[index, columnname]
    df_similarity.loc[index, "best_match_review_amount"] = df_similarity.loc[index, best_matches_review_amount[index]]
    df_similarity.loc[index, "best_match_cosine"] = df_similarity.loc[index, best_matches_similarity[index]]
    df_similarity.loc[index, "best_match_link"] = df_similarity.loc[index, best_matches_links[index]]
    df_similarity.loc[index, "best_match_type"] = best_matches_types[index][0]

df_similarity.head()

df_similarity["best_match_type"].value_counts()

,eikon_deal_id,date_announced,date_effective,acquiror,isin,full_name,full_name_cleaned,best_match_name,best_match_review_amount,best_match_cosine,best_match_link,best_match_type,full_name_legal_addon,link_hannes,link_hannes_nonstrict,link_marius,firm_hannes,firm_hannes_nonstrict,firm_marius,review_amount_hannes,review_amount_hannes_nonstrict,review_amount_marius,link_found_hannes,link_found_hannes_nonstrict,similarity_hannes,similarity_hannes_nonstrict,similarity_marius,similarity_mean,marius_equalto_hannes,date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction,total_assets,net_income,permid,nation,trbc_industry_group_code,trbc_industry_group_name,investment_acquiror,is_a_serial_acquiror_or_serial_target
0,34391733382,07/01/2008,04/03/2008,1,CA67072Q1046,NuVista Energy Ltd,NuVista Energy,NuVista Energy,8.0,1.00,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,hannes,Ltd,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,NaN,NuVista Energy,NuVista Energy,NaN,8.0,8.0,NaN,original review link,original review link,1.0,1.00,0.00,0.666667,0,31/12/2007,282962189,0.0,100.0,6.856333e+08,26422120.0,4.295861e+09,Canada,501020.0,Oil & Gas,0,0
1,34391733368,07/01/2008,05/03/2008,1,US4519061018,L-1 Identity Solutions Inc,L-1 Identity Solutions,Identity Solutions,NaN,0.83,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,nonstrict,Inc,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,NaN,NaN,Identity Solutions,NaN,NaN,NaN,NaN,no link found,transformed overview link,0.0,0.83,0.00,0.276667,1,31/12/2007,43970500,0.0,100.0,1.408352e+09,17666000.0,4.295904e+09,United States,521020.0,"Machinery, Tools, Heavy Vehicles, Trains & Ships",0,1
2,34391297298,08/01/2008,18/07/2008,1,US62845B1044,MutualFirst Financial Inc,MutualFirst Financial,Mutual of Omaha,805.0,0.36,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,nonstrict,Inc,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,NaN,NaN,Mutual of Omaha,NaN,NaN,805.0,NaN,no link found,transformed overview link,0.0,0.36,0.00,0.120000,1,31/12/2007,57168000,0.0,100.0,9.573430e+08,4226000.0,4.295914e+09,United States,551010.0,Banking Services,0,0
3,34391297367,09/01/2008,30/05/2008,1,US9841022026,Hampton Roads Bankshares Inc,Hampton Roads Bankshares,Hampton Roads Bankshares,NaN,1.00,https://www.glassdoor.com/Reviews/Hampton-Roads-Bankshares-Reviews-E24277.htm,hannes,Inc,https://www.glassdoor.com/Reviews/Hampton-Roads-Bankshares-Reviews-E24277.htm,https://www.glassdoor.com/Reviews/United-Bankshares-Reviews-E1987.htm,https://www.glassdoor.com/Reviews/Xenith-Bankshares-Reviews-E38765.htm,Hampton Roads Bankshares,United Bankshares,Xenith Bankshares,NaN,162.0,1.0,original review link,transformed overview link,1.0,0.35,0.31,0.553333,0,31/12/2007,56481000,0.0,100.0,5.611700e+08,6811000.0,4.295902e+09,United States,551010.0,Banking Services,0,1
4,34391297484,10/01/2008,31/10/2008,1,US2922181043,Employers Holdings Inc,Employers,EMPLOYERS,253.0,1.00,https://www.glassdoor.com/Reviews/EMPLOYERS-Reviews-E25702.htm,hannes,Holdings Inc,https://www.glassdoor.com/Reviews/EMPLOYERS-Reviews-E25702.htm,https://www.glassdoor.com/Reviews/EMPLOYERS-Reviews-E25702.htm,https://www.glassdoor.com/Reviews/Wells-Fargo-Reviews-E8876.htm,EMPLOYERS,EMPLOYERS,Wells Fargo,253.0,253.0,39130.0,original review link,original review link,1.0,1.00,0.00,0.666667,0,30/06/2008,190579000,0.0,100.0,3.081413e+09,114504000.0,4.295903e+09,United States,553010.0,Insurance,0,0


In [32]:
df_similarity.to_csv(Path.joinpath(Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample'), f'{time.strftime("%m%d%y")}_YahooAndBing_9496_complete.csv'), sep=";", decimal=",", index=False)

#  check how many suited deals

### get targets

In [37]:
df_similarity['date_announced'] = pd.to_datetime(df_similarity['date_announced'], format="%d/%m/%Y")

df_similarity_target = df_similarity[(df_similarity["acquiror"] == 0) & 
    (df_similarity["best_match_review_amount"] > 0) &
    (df_similarity["date_announced"].dt.year >= 2008)]

new_names = [(i,'target_'+i) for i in df_similarity_target.iloc[:, 4:].columns.values] # prefix
df_similarity_target = df_similarity_target.rename(columns = dict(new_names))

df_similarity_target.head()
df_similarity_target.shape

,eikon_deal_id,date_announced,acquiror,high_quality_observation,target_total_assets,target_full_name,target_full_name_cleaned,target_best_match_name,target_best_match_review_amount,target_best_match_link,target_best_match_cosine,target_full_name_legal_addon,target_permid,target_isin,target_is_a_serial_acquiror_or_serial_target,target_link_marius,target_link_hannes,target_link_hannes_nonstrict,target_firm_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_review_amount_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_link_found_hannes,target_link_found_hannes_nonstrict,target_cosine_marius,target_similarity_marius,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_mean,target_marius_equalto_hannes
4748,34391733382,2008-01-07,0,1,6.100595e+08,Rider Resources Ltd,Rider Resources,Rider Resources,1.0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,1.0,Ltd,4.295861e+09,CA7656982047,0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,Rider Resources,Rider Resources,Rider Resources,1.0,NaN,NaN,transformed overview link,transformed overview link,1.0,1.0,1.0,1.0,1.000000,1
4749,34391733368,2008-01-07,0,1,2.747402e+07,Bioscrypt Inc,Bioscrypt,Bioscrypt,2.0,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,1.0,Inc,4.295860e+09,CA09065U1066,0,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,Bioscrypt,Bioscrypt,Bioscrypt,2.0,2.0,2.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.0,1.000000,1
4750,34391297298,2008-01-08,0,1,5.137920e+08,MFB Corp,MFB,MFB,3.0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,1.0,Corp,4.295907e+09,US55272D1063,0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,"https://www.glassdoor.com/job-listing/occupational-therapist-mary-free-bed-rehabilitation-hospital-JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,MFB,"occupational therapist mary free bed rehabilitation hospital JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",MFB,3.0,NaN,3.0,wrong link,transformed overview link,1.0,1.0,0.0,1.0,0.666667,1
4751,34391297367,2008-01-09,0,1,2.666670e+08,Shore Financial Corp,Shore Financial,Shore Financial,2.0,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,1.0,Corp,4.295908e+09,US82511E1091,0,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,Shore Financial,Shore Financial,Shore Financial,2.0,NaN,NaN,transformed overview link,transformed overview link,1.0,1.0,1.0,1.0,1.000000,1
4752,34391297484,2008-01-10,0,1,6.359760e+08,AmCOMP Inc,AmCOMP,AmCOMP,1.0,https://www.glassdoor.com/Reviews/AmCOMP-Reviews-E8922.htm,1.0,Inc,4.295910e+09,US02342J1016,0,https://www.glassdoor.com/Reviews/AmCOMP-Reviews-E8922.htm,https://www.glassdoor.com/Reviews/AmCOMP-Reviews-E8922.htm,https://www.glassdoor.com/Reviews/AmCOMP-Reviews-E8922.htm,AmCOMP,AmCOMP,AmCOMP,1.0,1.0,1.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.0,1.000000,1


(3447, 32)

In [29]:
df_similarity[df_similarity["eikon_deal_id"] == 154083651144]

,eikon_deal_id,date_announced,acquiror,high_quality_observation,total_assets,full_name,full_name_cleaned,best_match_name,best_match_review_amount,best_match_link,best_match_cosine,full_name_legal_addon,permid,isin,is_a_serial_acquiror_or_serial_target,link_marius,link_hannes,link_hannes_nonstrict,firm_marius,firm_hannes,firm_hannes_nonstrict,review_amount_marius,review_amount_hannes,review_amount_hannes_nonstrict,link_found_hannes,link_found_hannes_nonstrict,cosine_marius,similarity_marius,similarity_hannes,similarity_hannes_nonstrict,similarity_mean,marius_equalto_hannes
1774,154083651144,2017-11-28,1,1,1.599600e+09,Cineworld Group PLC,Cineworld Group,Cineworld Cinemas,780.0,https://www.glassdoor.com/Reviews/Cineworld-Cinemas-Reviews-E686139.htm,0.60,PLC,4.295899e+09,GB00B15FWH70,0,NaN,https://www.glassdoor.com/Reviews/Cineworld-Cinemas-Reviews-E686139.htm,https://www.glassdoor.com/Reviews/Cineworld-Cinemas-Reviews-E686139.htm,,Cineworld Cinemas,Cineworld Cinemas,NaN,780.0,780.0,transformed overview link,transformed overview link,NaN,0.0,0.59,0.60,0.396667,0
6522,154083651144,2017-11-28,0,1,2.616000e+09,Regal Entertainment Group,Regal Entertainment Group,Regal Regal entertainment group,NaN,"https://www.glassdoor.com/Reviews/Regal-Regal-entertainment-group-Reviews-EI_IE14917.0,5_KO6,31.htm",0.91,NaN,4.295902e+09,US7587661098,0,NaN,https://www.glassdoor.com/Reviews/Regal-Reviews-E14917.htm,"https://www.glassdoor.com/Reviews/Regal-Regal-entertainment-group-Reviews-EI_IE14917.0,5_KO6,31.htm",,Regal,Regal Regal entertainment group,NaN,3500.0,NaN,transformed overview link,transformed jobs link,NaN,0.0,0.45,0.91,0.453333,0


### get acquirers

In [17]:
# get only acquirors with at least 1 review
df_similarity_acq = df_similarity[(df_similarity["acquiror"] == 1) & 
    (df_similarity["best_match_review_amount"] > 0)]

new_names = [(i,'acquiror_'+i) for i in df_similarity_acq.iloc[:, 4:].columns.values] # prefix
df_similarity_acq = df_similarity_acq.rename(columns = dict(new_names))

df_similarity_acq = df_similarity_acq.drop(labels=['date_announced','acquiror','high_quality_observation',], axis=1)

df_similarity_acq.head()
df_similarity_acq.shape

,eikon_deal_id,acquiror_total_assets,acquiror_full_name,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_review_amount,acquiror_best_match_link,acquiror_best_match_cosine,acquiror_full_name_legal_addon,acquiror_permid,...,acquiror_review_amount_hannes,acquiror_review_amount_hannes_nonstrict,acquiror_link_found_hannes,acquiror_link_found_hannes_nonstrict,acquiror_cosine_marius,acquiror_similarity_marius,acquiror_similarity_hannes,acquiror_similarity_hannes_nonstrict,acquiror_similarity_mean,acquiror_marius_equalto_hannes
0,34391733382,6.856333e+08,NuVista Energy Ltd,NuVista Energy,NuVista Energy,8.0,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,1.00,Ltd,4.295861e+09,...,8.0,8.0,original review link,original review link,NaN,0.00,1.00,1.00,0.666667,0
2,34391297298,9.573430e+08,MutualFirst Financial Inc,MutualFirst Financial,Mutual of Omaha,805.0,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,0.36,Inc,4.295914e+09,...,NaN,805.0,no link found,transformed overview link,NaN,0.00,0.00,0.36,0.120000,1
4,34391297484,3.081413e+09,Employers Holdings Inc,Employers,EMPLOYERS,253.0,https://www.glassdoor.com/Reviews/EMPLOYERS-Reviews-E25702.htm,1.00,Holdings Inc,4.295903e+09,...,253.0,253.0,original review link,original review link,NaN,0.00,1.00,1.00,0.666667,0
5,34391297451,1.720000e+12,Bank of America Corp,Bank of America,Bank of America,32800.0,https://www.glassdoor.com/Reviews/Bank-of-America-Reviews-E8874.htm,1.00,Corp,8.589934e+09,...,32800.0,32800.0,transformed overview link,transformed overview link,NaN,0.00,1.00,1.00,0.666667,0
6,34391297533,6.173200e+09,Terex Corp,Terex,Terex Corporation,762.0,https://www.glassdoor.com/Reviews/Terex-Corporation-Reviews-E1954.htm,0.47,Corp,4.295905e+09,...,762.0,762.0,transformed overview link,transformed overview link,1.0,0.45,0.46,0.47,0.460000,1


(3971, 29)

### merge

In [25]:
df_merge = df_similarity_target.merge(df_similarity_acq, how="left", on="eikon_deal_id")

df_merge_filtered = df_merge[(df_merge["acquiror"] >= 0) & 
    # (df_merge["target_best_match_review_amount"] > 0) &
    # (df_merge["acquiror_best_match_review_amount"] > 0) &
    (df_merge["date_announced"].dt.year >= 2008)]

df_merge_filtered.head()
df_merge_filtered.shape

,eikon_deal_id,date_announced,acquiror,high_quality_observation,target_total_assets,target_full_name,target_full_name_cleaned,target_best_match_name,target_best_match_review_amount,target_best_match_link,target_best_match_cosine,target_full_name_legal_addon,target_permid,target_isin,target_is_a_serial_acquiror_or_serial_target,target_link_marius,target_link_hannes,target_link_hannes_nonstrict,target_firm_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_review_amount_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_link_found_hannes,...,acquiror_best_match_name,acquiror_best_match_review_amount,acquiror_best_match_link,acquiror_best_match_cosine,acquiror_full_name_legal_addon,acquiror_permid,acquiror_isin,acquiror_is_a_serial_acquiror_or_serial_target,acquiror_link_marius,acquiror_link_hannes,acquiror_link_hannes_nonstrict,acquiror_firm_marius,acquiror_firm_hannes,acquiror_firm_hannes_nonstrict,acquiror_review_amount_marius,acquiror_review_amount_hannes,acquiror_review_amount_hannes_nonstrict,acquiror_link_found_hannes,acquiror_link_found_hannes_nonstrict,acquiror_cosine_marius,acquiror_similarity_marius,acquiror_similarity_hannes,acquiror_similarity_hannes_nonstrict,acquiror_similarity_mean,acquiror_marius_equalto_hannes
0,34391733382,2008-07-01,0,1,6.100595e+08,Rider Resources Ltd,Rider Resources,Rider Resources,1.0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,1.0,Ltd,4.295861e+09,CA7656982047,0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,Rider Resources,Rider Resources,Rider Resources,1.0,NaN,NaN,transformed overview link,...,NuVista Energy,8.0,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,1.00,Ltd,4.295861e+09,CA67072Q1046,0.0,NaN,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,,NuVista Energy,NuVista Energy,NaN,8.0,8.0,original review link,original review link,NaN,0.0,1.0,1.00,0.666667,0.0
1,34391733368,2008-07-01,0,1,2.747402e+07,Bioscrypt Inc,Bioscrypt,Bioscrypt,2.0,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,1.0,Inc,4.295860e+09,CA09065U1066,0,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,Bioscrypt,Bioscrypt,Bioscrypt,2.0,2.0,2.0,transformed overview link,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34391297298,2008-08-01,0,1,5.137920e+08,MFB Corp,MFB,MFB,3.0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,1.0,Corp,4.295907e+09,US55272D1063,0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,"https://www.glassdoor.com/job-listing/occupational-therapist-mary-free-bed-rehabilitation-hospital-JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,MFB,"occupational therapist mary free bed rehabilitation hospital JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",MFB,3.0,NaN,3.0,wrong link,...,Mutual of Omaha,805.0,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,0.36,Inc,4.295914e+09,US62845B1044,0.0,NaN,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,,,Mutual of Omaha,NaN,NaN,805.0,no link found,transformed overview link,NaN,0.0,0.0,0.36,0.120000,1.0
3,34391297367,2008-09-01,0,1,2.666670e+08,Shore Financial Corp,Shore Financial,Shore Financial,2.0,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,1.0,Corp,4.295908e+09,US82511E1091,0,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,https://www.glassdoor.com/Reviews/Shore-Financial-Reviews-E5858286.htm,Shor

(3447, 60)

In [66]:
## save file with dynamic file name
dir = Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample')
path = Path.joinpath(dir, f'{time.strftime("%m%d%y")}_Eikon_2911deals.csv')
df_merge_filtered.to_csv(path, sep=";", decimal=",", index=False)
print(f"{path} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\110422_Eikon_2911deals.csv saved


## set filter criteria

In [11]:
reviews = [i for i in range(30,60,10)]
years = [i for i in range(2008, 2013, 1)]
cosine_tresholds = list(map(lambda x: x/10.0, range(5, 11, 1))) #range doesnt work with floats

for review in reviews:
    for yearr in years:
        for cosine_treshold in cosine_tresholds:
            df_merge_filtered = df_merge[
                (df_merge["acquiror"] == 0) & 
                (df_merge["target_best_match_review_amount"] >= review) &
                (df_merge["acquiror_best_match_review_amount"] >= review) &
                (df_merge["target_best_match_cosine"] >= cosine_treshold) &
                (df_merge["acquiror_best_match_cosine"] >= cosine_treshold) &
                (df_merge["date_announced"].dt.year >= yearr) &
                (df_merge["high_quality_observation"] >= 1)
                ]

            print(review, cosine_treshold, yearr,":",df_merge_filtered.shape[0])

30 0.5 2008 : 287
30 0.6 2008 : 253
30 0.7 2008 : 214
30 0.8 2008 : 186
30 0.9 2008 : 154
30 1.0 2008 : 147
30 0.5 2009 : 278
30 0.6 2009 : 246
30 0.7 2009 : 207
30 0.8 2009 : 183
30 0.9 2009 : 152
30 1.0 2009 : 145
30 0.5 2010 : 264
30 0.6 2010 : 232
30 0.7 2010 : 195
30 0.8 2010 : 175
30 0.9 2010 : 145
30 1.0 2010 : 138
30 0.5 2011 : 253
30 0.6 2011 : 222
30 0.7 2011 : 189
30 0.8 2011 : 170
30 0.9 2011 : 141
30 1.0 2011 : 134
30 0.5 2012 : 242
30 0.6 2012 : 211
30 0.7 2012 : 181
30 0.8 2012 : 164
30 0.9 2012 : 136
30 1.0 2012 : 129
40 0.5 2008 : 248
40 0.6 2008 : 218
40 0.7 2008 : 180
40 0.8 2008 : 157
40 0.9 2008 : 131
40 1.0 2008 : 124
40 0.5 2009 : 240
40 0.6 2009 : 212
40 0.7 2009 : 174
40 0.8 2009 : 154
40 0.9 2009 : 129
40 1.0 2009 : 122
40 0.5 2010 : 230
40 0.6 2010 : 202
40 0.7 2010 : 166
40 0.8 2010 : 148
40 0.9 2010 : 124
40 1.0 2010 : 117
40 0.5 2011 : 221
40 0.6 2011 : 193
40 0.7 2011 : 161
40 0.8 2011 : 144
40 0.9 2011 : 121
40 1.0 2011 : 114
40 0.5 2012 : 211
40 0.6 201

# examples

## manually vectorize strings & calc cosine similarity

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def text_similarity(row, column_name1, column_name2):
    ## vectorize strings to word count lists
    count_vect = CountVectorizer(lowercase=True) #object needs to be initialized for the (optional) df later on
    texts = [row[column_name1],row[column_name2]]
    vectorized_texts_object = count_vect.fit_transform(texts)
    vectorized_texts = vectorized_texts_object.toarray() #2 list entries

    ## calc cosine similarity (0-1; with 1 being perfectly identical)
    similarity = cosine_similarity([vectorized_texts[0]], [vectorized_texts[1]])
    similarity_score = round(float(similarity),3)
    return similarity_score

In [ ]:
df["Max_Cosine_Similarity_Hannes"] = df.apply(text_similarity, args=("company_name", "company_name"), axis=1) #args to set both column names that should be used
df

### example: vectorize strings & calc cosine similarity

In [14]:
def text_similarity(text1, text2):
    ## vectorize strings to word count lists
    count_vect = CountVectorizer(lowercase=True) #object needs to be initialized for the (optional) df later on
    texts = [text1,text2]
    vectorized_texts_object = count_vect.fit_transform(texts)
    vectorized_texts = vectorized_texts_object.toarray() #2 list entries

    df_vectorized_texts = pd.DataFrame(vectorized_texts_object.toarray(),columns=count_vect.get_feature_names_out(),index=["text1", "text2"])
    print(df_vectorized_texts)

    # print(vectorized_texts[0])
    # print(vectorized_texts[1])

    ## calc cosine similarity (0-1; with 1 being perfectly identical)
    similarity = cosine_similarity([vectorized_texts[0]], [vectorized_texts[1]])
    similarity_score = round(float(similarity),3)
    return similarity_score


text1 = "Champignon Brands"
text2 = "Champion Brands"
text_similarity(text1, text2)

       brands  champignon  champion
text1       1           1         0
text2       1           0         1


0.5

#### Levinshtein Distance

In [10]:
from fuzzywuzzy import fuzz
# df['score'] = df[['Name Left','Name Right']].apply(lambda x : fuzz.partial_ratio(*x),axis=1)

text1 = "Champignon Brands"
text2 = "Champion Brands"
fuzz.partial_ratio(text1, text2)

87

### String_Grouper Module

In [11]:
from string_grouper import compute_pairwise_similarities

# Create a small DataFrame of pairs of strings:
pair_s = pd.DataFrame(
    [
        ('Mega Enterprises Corporation', 'Mega Enterprises Corporation'),
        ('Hyper Startup Inc.', 'Hyper Startup Incorporated'),
        ('Hyper Startup Inc.', 'Hyper Startup Inc.'),
        ('Hyper Startup Inc.', 'Hyper-Startup Inc.'),
        ('Hyper Hyper Inc.', 'Hyper Hyper Inc.'),
        ('Mega Enterprises Corporation', 'Mega Enterprises Corp.')
   ],
   columns=('left', 'right')
)

pair_s['similarity'] = compute_pairwise_similarities(pair_s['left'], pair_s['right'])
pair_s

,left,right,similarity
0,Mega Enterprises Corporation,Mega Enterprises Corporation,1.000000
1,Hyper Startup Inc.,Hyper Startup Incorporated,0.633620
2,Hyper Startup Inc.,Hyper Startup Inc.,1.000000
3,Hyper Startup Inc.,Hyper-Startup Inc.,1.000000
4,Hyper Hyper Inc.,Hyper Hyper Inc.,1.000000
5,Mega Enterprises Corporation,Mega Enterprises Corp.,0.826463
